In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from src.analyse import test_trans
from src.analyse import analyse

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200
plt.rcParams['figure.figsize'] = [10, 4]
plt.rcParams['figure.dpi'] = 100

pd.__version__

'1.1.3'

In [150]:
missing_values = ["n/a", "na", "--"]
train = pd.read_csv("data/train.csv", na_values = missing_values)
test = pd.read_csv("data/test.csv", na_values = missing_values)
# train.head(20)
# train.shape

In [151]:
# Split features and labels
train_labels = train['SalePrice'].reset_index(drop=True)
train_features = train.drop(['SalePrice'], axis=1)
test_features = test

# Combine train and test features in order to apply the feature transformation pipeline to the entire dataset
all_features = pd.concat([train_features, test_features]).reset_index(drop=True)
all_features.shape



(2919, 80)

# Feature Engineering

Before we proceed with fitting the model, we will need to make adjustments to the features to make it suitable for the model. 

- Handling missing values.

In [33]:
# Combining features from the train and test dataset in order to perform adjustments to both datasets. 

train_features = train.drop(["SalePrice"], axis = 1)
test_features = test

all_features = pd.concat([train_features, test_features]).reset_index(drop = True)
all_features
all_features.shape

(2919, 80)

In [5]:
all_features_nulls = all_features.isnull().sum() # returns number of null values for each feature. 
all_features_nulls[all_features_nulls > 0].sort_values() # only returns columns with missing values and the number of missing values for each of those columns. 
len(all_features_nulls[all_features_nulls > 0]) # number of features with null values. 

train_features_nulls = train_features.isnull().sum() # returns number of null values for each column in the train dataset. 
len(train_features_nulls[train_features_nulls > 0]) 
train_features_nulls

test_features_nulls = test_features.isnull().sum()
len(test_features_nulls[test_features_nulls > 0])

print(f"""Number of columns with missing values: 
Train features: {len(train_features_nulls[train_features_nulls > 0])}
Test features: {len(test_features_nulls[test_features_nulls > 0])}
All features: {len(all_features_nulls[all_features_nulls > 0])}""")

Number of columns with missing values: 
Train features: 19
Test features: 33
All features: 34


In [6]:
all_features_nulls[all_features_nulls > 0].sort_values(ascending = False)
# all_features.shape

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
FireplaceQu     1420
LotFrontage      486
GarageFinish     159
GarageYrBlt      159
GarageQual       159
GarageCond       159
GarageType       157
BsmtExposure      82
BsmtCond          82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MasVnrArea        23
MSZoning           4
BsmtFullBath       2
BsmtHalfBath       2
Utilities          2
Functional         2
Exterior2nd        1
Exterior1st        1
SaleType           1
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
Electrical         1
KitchenQual        1
GarageCars         1
GarageArea         1
TotalBsmtSF        1
dtype: int64

In [7]:
# len(all_features[all_features["BsmtFinSF2"] == 0]["BsmtFinSF2"])
# len(all_features[all_features["BsmtFinSF1"] == 0]["BsmtFinSF1"])

len(all_features[all_features["BsmtUnfSF"] == 0]["BsmtUnfSF"])
# all_features[all_features["BsmtUnfSF"].isna()]["BsmtUnfSF"]

241

In [8]:
all_features[all_features["Functional"].isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
2216,2217,20,NaN,80.0,14584,Pave,NaN,Reg,Low,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1Story,1,5,1952,1952,Gable,CompShg,AsbShng,VinylSd,None,0.0,Fa,Po,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,Wall,Po,N,FuseA,733,0,0,733,0.0,0.0,1,0,2,1,Fa,4,NaN,0,NaN,Attchd,1952.0,Unf,2.0,487.0,Fa,Po,N,0,0,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Abnorml
2473,2474,50,RM,60.0,10320,Pave,Grvl,Reg,Lvl,AllPub,Corner,Gtl,IDOTRR,Artery,Norm,1Fam,1.5Fin,4,1,1910,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Fa,Fa,CBlock,TA,Fa,No,Unf,0.0,Unf,0.0,771.0,771.0,GasA,Fa,Y,SBrkr,866,504,114,1484,0.0,0.0,2,0,3,1,TA,6,NaN,0,NaN,Detchd,1910.0,Unf,1.0,264.0,TA,Fa,N,14,211,0,0,84,0,NaN,NaN,NaN,0,9,2007,COD,Abnorml


In [9]:
null_cols = all_features.columns[all_features.isnull().any()] # returns list of columns with null values. 
all_features[null_cols] # returns df consisting only of columns with null values. 

all_features[null_cols].isnull().all(axis = 1) # boolean: returns True for rows with missing values for all columns. 

all_features[null_cols][all_features[null_cols].isnull().all(axis  = 1)] # returns df with rows which have null valules for all columns. 

,MSZoning,LotFrontage,Alley,Utilities,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Electrical,BsmtFullBath,BsmtHalfBath,KitchenQual,Functional,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PoolQC,Fence,MiscFeature,SaleType


In [10]:
# mask = all_features.isnull().sum()  # boolean. columns with less than 5 missing values. 
# # all_features[null_cols][all_features[null_cols].isnull().sum() < 5]
# mask[mask < 5] & mask[mask != 0]


mask = all_features[null_cols].isnull().sum() # number of missing values for columns with missing values. 
null_cols_5 = list(mask[mask < 5].index) # list of columns with less than 5 missing values. 
null_cols_2 = list(mask[mask < 2].index) # list of columns with less than 2 missing values. 

# all_features[null_cols_5][all_features[null_cols_5].isnull().all(axis = 0)]
all_features[null_cols_2][all_features[null_cols_2].isnull().all(axis = 1)] # returns df with rows that have missing values for all columns. 


all_features[null_cols_2][all_features[null_cols_2]["TotalBsmtSF"] == 0]
len(all_features[null_cols_2][all_features[null_cols_2]["TotalBsmtSF"] == 0]) # number of rows in TotalBsmtSF column with value 0.0






78

In [11]:
all_features[all_features["GarageArea"].isnull()]
len(all_features[all_features["GarageArea"] == 0])

all_features[null_cols].iloc[2120]

MSZoning             RM
LotFrontage          99
Alley               NaN
Utilities        AllPub
Exterior1st     MetalSd
Exterior2nd      CBlock
MasVnrType         None
MasVnrArea            0
BsmtQual            NaN
BsmtCond            NaN
BsmtExposure        NaN
BsmtFinType1        NaN
BsmtFinSF1          NaN
BsmtFinType2        NaN
BsmtFinSF2          NaN
BsmtUnfSF           NaN
TotalBsmtSF         NaN
Electrical        FuseA
BsmtFullBath        NaN
BsmtHalfBath        NaN
KitchenQual          TA
Functional          Typ
FireplaceQu         NaN
GarageType       Detchd
GarageYrBlt        1946
GarageFinish        Unf
GarageCars            1
GarageArea          280
GarageQual           TA
GarageCond           TA
PoolQC              NaN
Fence             MnPrv
MiscFeature         NaN
SaleType          ConLD
Name: 2120, dtype: object

In [36]:
# mask1 = all_features[null_cols]["GarageFinish"].isnull() & all_features[null_cols]["GarageType"].notnull()
# null_cols
# all_features[mask1]
# all_features.loc[2576]

# all_features[mask1].replace("Detchd", np.nan)

# all_features[mask1]["GarageType"] = all_features[mask1]["GarageType"].replace("Detchd", np.nan)

# all_features.loc[mask1, "GarageType"] =   all_features.loc[mask1, "GarageType"].replace("Detchd", np.nan)
# all_features

# len(all_features[all_features["GarageType"].isnull()])

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
2126,2127,60,RM,57.0,8094,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2.5Unf,6,8,1910,1983,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,PConc,TA,TA,Mn,Rec,196.0,Unf,0.0,1046.0,1242.0,GasA,Gd,Y,SBrkr,1242,742,0,1984,0.0,0.0,2,0,5,1,TA,8,Typ,0,NaN,Detchd,NaN,NaN,1.0,360.0,NaN,NaN,Y,64,0,180,0,0,0,NaN,MnPrv,Shed,1000,9,2008,WD,Normal
2576,2577,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,2Story,5,6,1923,1999,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,BrkTil,Gd,TA,No,ALQ,548.0,Unf,0.0,311.0,859.0,GasA,Ex,Y,SBrkr,942,886,0,1828,0.0,0.0,2,0,3,1,Gd,6,Typ,0,NaN,Detchd,NaN,NaN,NaN,NaN,NaN,NaN,Y,174,0,212,0,0,0,NaN,MnPrv,NaN,0,3,2007,WD,Alloca


In [32]:
del all_features

In [109]:
mask1 = all_features["GarageFinish"].isnull() # mask to only return rows which have a nan value for GarageFinish column. 
# all_features[all_features.filter(like = 'Garage').columns] # returns df only consisting of columns containing the word Garage. 

# all_features[mask1][all_features.filter(like= 'Garage').columns].replace(to_replace = {"GarageType" : {: np.nan},})
# all_features[mask1][all_features.filter(like= 'Garage').columns]

# all_features[mask1].replace(to_replace = {"GarageType": {}})
# all_features_gn = all_features[mask1][all_features.filter(like= 'Garage').columns]
# all_features_gn.loc[all_features_gn["GarageType"].notnull(), "GarageType"] = np.nan
# all_features.loc[all_features["GarageType"].notnull(), "GarageType"] = np.nan
# all_features.loc[all_features[]]
# all_features.loc[2126]
# df.loc[df['note'].notnull(), 'note'] = 1

all_features[mask1].loc[all_features[mask1].GarageType.notnull(), "GarageType"] = np.nan
# mask1

all_features[mask1].GarageType

all_features[mask1].loc[all_features[mask1]["GarageType"].notnull(), "GarageType"] = 5
# all_features[mask1].loc[all_features[mask1].GarageType.notnull(), "GarageType"]
all_features[mask1].GarageType



39         NaN
48         NaN
78         NaN
88         NaN
89         NaN
99         NaN
108        NaN
125        NaN
127        NaN
140        NaN
148        NaN
155        NaN
163        NaN
165        NaN
198        NaN
210        NaN
241        NaN
250        NaN
287        NaN
291        NaN
307        NaN
375        NaN
386        NaN
393        NaN
431        NaN
434        NaN
441        NaN
464        NaN
495        NaN
520        NaN
528        NaN
533        NaN
535        NaN
562        NaN
582        NaN
613        NaN
614        NaN
620        NaN
635        NaN
636        NaN
638        NaN
649        NaN
705        NaN
710        NaN
738        NaN
750        NaN
784        NaN
826        NaN
843        NaN
921        NaN
942        NaN
954        NaN
960        NaN
968        NaN
970        NaN
976        NaN
1009       NaN
1011       NaN
1030       NaN
1038       NaN
1096       NaN
1123       NaN
1131       NaN
1137       NaN
1143       NaN
1173       NaN
1179      

In [137]:
# all_features2 = all_features[mask1].copy()

# # all_features2.loc[all_features2.GarageType.notnull(), "GarageType"] = np.nan
# # all_features.loc[all_features[mask1].GarageType.notnull(), "GarageType"] = np.nan
# # all_features.loc[all_features[mask1].GarageType.notnull(), "GarageType"] = np.nan

# all_features[mask1].GarageType.notnull()



39      False
48      False
78      False
88      False
89      False
99      False
108     False
125     False
127     False
140     False
148     False
155     False
163     False
165     False
198     False
210     False
241     False
250     False
287     False
291     False
307     False
375     False
386     False
393     False
431     False
434     False
441     False
464     False
495     False
520     False
528     False
533     False
535     False
562     False
582     False
613     False
614     False
620     False
635     False
636     False
638     False
649     False
705     False
710     False
738     False
750     False
784     False
826     False
843     False
921     False
942     False
954     False
960     False
968     False
970     False
976     False
1009    False
1011    False
1030    False
1038    False
1096    False
1123    False
1131    False
1137    False
1143    False
1173    False
1179    False
1218    False
1219    False
1234    False
1257    False
1283  

In [164]:
all_features.loc[2126, "GarageType"] = np.nan
all_features.loc[2126, "GarageCars"] = np.nan
all_features.loc[2126, "GarageArea"] = np.nan
# all_features.loc[2576, "GarageType"] = np.nan

all_features.loc[2576]

Id                  2577
MSSubClass            70
MSZoning              RM
LotFrontage           50
LotArea             9060
Street              Pave
Alley                NaN
LotShape             Reg
LandContour          Lvl
Utilities         AllPub
LotConfig         Inside
LandSlope            Gtl
Neighborhood      IDOTRR
Condition1          Norm
Condition2          Norm
BldgType            1Fam
HouseStyle        2Story
OverallQual            5
OverallCond            6
YearBuilt           1923
YearRemodAdd        1999
RoofStyle          Gable
RoofMatl         CompShg
Exterior1st      Wd Sdng
Exterior2nd      Plywood
MasVnrType          None
MasVnrArea             0
ExterQual             TA
ExterCond             TA
Foundation        BrkTil
BsmtQual              Gd
BsmtCond              TA
BsmtExposure          No
BsmtFinType1         ALQ
BsmtFinSF1           548
BsmtFinType2         Unf
BsmtFinSF2             0
BsmtUnfSF            311
TotalBsmtSF          859
Heating             GasA


In [149]:
del all_features

In [16]:
mask2 = all_features[null_cols]["BsmtExposure"].isnull() & all_features[null_cols]["BsmtFinType1"].notnull()
# null_cols
all_features[mask2]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
948,949,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,144.0,Gd,TA,PConc,Gd,TA,NaN,Unf,0.0,Unf,0.0,936.0,936.0,GasA,Ex,Y,SBrkr,936,840,0,1776,0.0,0.0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2002.0,RFn,2.0,474.0,TA,TA,Y,144,96,0,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Normal
1487,1488,20,RL,73.0,8987,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2005,2006,Gable,CompShg,VinylSd,VinylSd,BrkFace,226.0,Gd,TA,PConc,Gd,TA,NaN,Unf,0.0,Unf,0.0,1595.0,1595.0,GasA,Ex,Y,SBrkr,1595,0,0,1595,0.0,0.0,2,0,2,1,Gd,6,Typ,1,Gd,Attchd,2005.0,RFn,3.0,880.0,TA,TA,Y,144,0,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
2348,2349,60,FV,81.0,10411,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,Somerst,Norm,Norm,1Fam,2Story,5,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,CBlock,Gd,TA,NaN,Unf,0.0,Unf,0.0,725.0,725.0,GasA,Ex,Y,SBrkr,725,863,0,1588,0.0,0.0,3,0,3,1,Gd,8,Typ,0,NaN,Attchd,2007.0,Unf,2.0,561.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2007,New,Partial


In [11]:
###############################
nulls = train.isnull().sum()
# nulls[nulls > 0]
# len(null_columns[null_columns > 0])
# null_columns
null_cols = nulls[nulls>0].sort_values() # Number of null values for variables with null values.
# null_var = list(null_cols.index) # List of variables with null/ missing values.
# train[null_var].dtypes # Data types of the variables with null values. 

# len(nulls[nulls > 0]) # Number of columns with null/ missing values. 

In [193]:
train["YrSold"]

0       2008
1       2007
2       2008
3       2006
4       2008
        ... 
1455    2007
1456    2010
1457    2010
1458    2010
1459    2008
Name: YrSold, Length: 1460, dtype: int64

## POOL VARIABLES 

- PoolArea
- PoolQC

In [169]:
train[["PoolArea", "PoolQC"]].info()

# No null values for PoolArea, however, there are only 7 non-null rows for the PoolQC column. Upon closer inspection, an input of 0 is 
# assigned for houses with no pool. There are only 7 non-zero entries for PoolArea which matches with the 7 non-null entries for PoolQC.

train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PoolArea  1460 non-null   int64 
 1   PoolQC    7 non-null      object
dtypes: int64(1), object(1)
memory usage: 22.9+ KB


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [44]:
pa_notnull = train["PoolArea"] != 0
train[pa_notnull][["PoolArea", "PoolQC"]]



,PoolArea,PoolQC
197,512,Ex
810,648,Fa
1170,576,Gd
1182,555,Ex
1298,480,Gd
1386,519,Fa
1423,738,Gd


In [51]:
poolqc = train[["PoolArea", "PoolQC"]].groupby("PoolQC")
poolqc.mean()

# There does not seem to be a clear correlation between the PoolArea and PoolQC. Originally, my assumption was that the PoolQC will be 
# higher for larger PoolArea. 

,PoolArea
PoolQC,
Ex,533.5
Fa,583.5
Gd,598.0


## Basement Variables

There are 11 variables related to the basement:
- BsmtQual
- BsmtCond
- BsmtExposure
- BsmtFinType1
- BsmtFinSF1
- BsmtFinType2
- BsmtFinSF2
- BsmtUnfSF
- TotalBsmtSF
- BsmtFullBath
- BsmtHalfBath

In [65]:
train.columns
bsmt_var = ['BsmtQual','BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']

train_bsmt = train[bsmt_var]

print(train_bsmt.isnull().sum())

BsmtQual        37
BsmtCond        37
BsmtExposure    38
BsmtFinType1    37
BsmtFinSF1       0
BsmtFinType2    38
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
BsmtFullBath     0
BsmtHalfBath     0
dtype: int64

In [85]:
x = train_bsmt["BsmtQual"].isnull()

train_bsmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BsmtQual      1423 non-null   object
 1   BsmtCond      1423 non-null   object
 2   BsmtExposure  1422 non-null   object
 3   BsmtFinType1  1423 non-null   object
 4   BsmtFinSF1    1460 non-null   int64 
 5   BsmtFinType2  1422 non-null   object
 6   BsmtFinSF2    1460 non-null   int64 
 7   BsmtUnfSF     1460 non-null   int64 
 8   TotalBsmtSF   1460 non-null   int64 
 9   BsmtFullBath  1460 non-null   int64 
 10  BsmtHalfBath  1460 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 125.6+ KB


In [96]:
len(train_bsmt[train_bsmt["BsmtFinSF1"] == 0]["BsmtFinSF1"])

467

In [97]:
len(train_bsmt[train_bsmt["BsmtFinSF1"] != 0])

993

In [105]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [98]:
for column in train.columns:
    if train[column].dtype == 'object':
        train[column] = train[column].astype("category")
    else:
        pass

In [104]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1         int64
BsmtFinType2      object
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating           object
